In [8]:
# %load_ext autoreload
# %autoreload 2
# %matplotlib inline

In [9]:
# from matplotlib import pyplot as plt 
# from matplotlib import figure, colors
# from matplotlib.patches import Rectangle
#from data import visualize, visualize_multiple
#import PIL.ExifTags
#from PIL import ImageOpsimport xmltodict
#from torchvision.transforms import functional as F
#from engine import train_one_epoch

# import os
# import numpy as np
# import torch
# import torch.utils.data
# from PIL import Image
# import random
# from model import fasterrcnn_resnet50_fpn
# import utils.transforms, utils.datasets, utils.optimizer

In [10]:
handle = '/output'
#handle = './output'

In [11]:
from data import ID_TO_NAME_MAP_EROSION as ID_TO_NAME_MAP, NAME_TO_ID_MAP_EROSION as NAME_TO_ID_MAP
import torchvision.transforms as transforms
import torch.nn as nn
import torchvision
import torch
import os
from PIL import Image
import numpy as np
import pandas as pd

In [12]:
import logging
logger = logging.getLogger('0')
hdlr = logging.FileHandler(handle  +'/logs/4-erosionscore.log')
formatter = logging.Formatter('[%(asctime)s][%(levelname)s]   %(message)s')
hdlr.setFormatter(formatter)
logger.addHandler(hdlr) 
logger.setLevel(logging.INFO)
consoleHandler = logging.StreamHandler()
consoleHandler.setFormatter(formatter)
logger.addHandler(consoleHandler)

In [13]:
TEST = '/test/'
PATH = './training.csv'
JOINTINPUT = handle  +'/test/erosion_all'
PRETRAIN_MODEL = './pretrained_models/narrow_score_75.pth'#'./pretrained_model/narrow_score_80_with_batchnormfull_scale_50.pth'
SCOREOUTPUT = handle  +'/erosion_score.csv'
logger.info (f'INPUT:{JOINTINPUT}, OUTPUT:{SCOREOUTPUT}, PRETRAIN_MODEL:{PRETRAIN_MODEL}')

[2020-05-09 13:57:24,458][INFO]   INPUT:./output/test/erosion_all, OUTPUT:./output/erosion_score.csv, PRETRAIN_MODEL:./pretrained_models/narrow_score_75.pth


In [14]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
print(device)

cuda


### Data

In [15]:
class scale(object):
    def __call__(self, image):
        image = image/image.mean()*0.5
        return image
    
class ScoreDataset(torch.utils.data.Dataset):
    def __init__(self, data_path, transform=None):
        self.transforms = transform
        self.img_folder = data_path 
        self.imgs = list(sorted(os.listdir(self.img_folder)))

    def __getitem__(self, idx):
        img = Image.open(os.path.join(self.img_folder,self.imgs[idx])).convert("RGB")
        target = {}
        image_id = torch.tensor([idx])
        target["image_id"] = image_id
        if self.transforms is not None:
            img = self.transforms(img)
        return img

    def __len__(self):
        return len(self.imgs)

In [16]:
transform = transforms.Compose(   
    [transforms.Resize((128,128)),
    transforms.ToTensor(),
    scale(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

dataset = ScoreDataset(JOINTINPUT, transform = transform)
dataloader = torch.utils.data.DataLoader(dataset, batch_size=128, num_workers=2, shuffle = False)
logger.info(f'Length of dataset: {len(dataloader)}')

[2020-05-09 13:57:27,879][INFO]   Length of dataset: 1


In [17]:
class Base_Classification_Net2(nn.Module):
    def __init__(self, num_classes):
        super(Base_Classification_Net2, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 16, 3, padding = 1),
            nn.BatchNorm2d(16),
            nn.ReLU(True),
            nn.MaxPool2d(2, 2),
            nn.Conv2d(16, 64, 3, padding = 1),
            nn.BatchNorm2d(64),
            nn.ReLU(True),
            nn.MaxPool2d(2, 2),
            nn.Conv2d(64, 256, 3),
            nn.BatchNorm2d(256),
            nn.ReLU(True),
            nn.MaxPool2d(2, 2),
            nn.Conv2d(256, 1024, 3, stride = 3),
            nn.BatchNorm2d(1024),
            nn.ReLU(True),
        )
        self.classifier = nn.Sequential(
            nn.Linear(1024 * 5 * 5, 256),
            nn.ReLU(True),
            nn.BatchNorm1d(256),
            nn.Linear(256, 64),
            nn.ReLU(True),
            nn.BatchNorm1d(64),
            nn.Linear(64, num_classes)
        )
        
    def forward(self, x):
        x = self.features(x)
        x = torch.flatten(x, 1)
        x = self.classifier(x)
        return x

In [18]:
net = Base_Classification_Net2(num_classes = 5)
net.to(device)
net.load_state_dict(torch.load(PRETRAIN_MODEL))

<All keys matched successfully>

In [19]:
def validate(model,validloader, device):
    correct = 0
    total = 0
    all_predictions = []
    with torch.no_grad():
        for i,images in enumerate(validloader):
            logger.info(f'Inference on batch {i}')
            images = images.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += predicted.size(0)
            all_predictions.append(predicted)
    all_predictions = np.array([float(x) for x in torch.cat(all_predictions,0).cpu()])
    return all_predictions

In [20]:
predictions = validate(net, dataloader, device)

[2020-05-09 13:57:34,077][INFO]   Inference on batch 0


In [21]:
assert len(predictions) == len(dataset)

In [26]:
output = []
for i,x in enumerate(dataset.imgs):
    x = x.strip('.jpg').split('-')
    name = ID_TO_NAME_MAP[float(x[2])].split('_')
    name.insert(2, 'E_')
    x[2] = '_'.join(name)
    if x[1] in x[2]:
        x.pop(1)
        x.append(predictions[i])
        output.append(x)
    else:
        logger.error(f'There is a mislabel {x[2]} for image {dataset.imgs[i]}')
output = pd.DataFrame(output, columns = ['Patient_ID', 'Joint', 'Score'])
pivoted = output.pivot(index = 'Patient_ID', columns = 'Joint', values = 'Score')
logger.warning(f'Number of missing per joint: {dict(pivoted.isna().sum()[pivoted.isna().sum()>0])}')
pivoted = pivoted.reset_index().sort_values(by = 'Patient_ID',ascending = True)

[2020-05-09 13:59:33,156][ERROR]   There is a mislabel LH_wrist_E__ulna for image UAB001-RF-22.jpg
[2020-05-09 13:59:33,158][ERROR]   There is a mislabel LH_wrist_E__ulna for image UAB002-RF-22.jpg
[2020-05-09 13:59:33,166][WARNING]   Number of missing per joint: {}


In [34]:
output

,Patient_ID,Joint,Score
0,UAB001,LF_mtp_E__1,0.0
1,UAB001,LF_mtp_E__2,0.0
2,UAB001,LF_mtp_E__3,0.0
3,UAB001,LF_mtp_E__4,2.0
4,UAB001,LF_mtp_E__5,3.0
...,...,...,...
81,UAB002,RH_wrist_E__mc1,0.0
82,UAB002,RH_wrist_E__mul,0.0
83,UAB002,RH_wrist_E__nav,0.0
84,UAB002,RH_wrist_E__radius,0.0


In [28]:
pivoted.T

,0,1
Joint,,
Patient_ID,UAB001,UAB002
LF_mtp_E__1,0,0
LF_mtp_E__2,0,0
LF_mtp_E__3,0,0
LF_mtp_E__4,2,0
LF_mtp_E__5,3,0
LF_mtp_E__ip,0,0
LH_mcp_E__1,0,0
LH_mcp_E__2,1,0


In [22]:
Patient_ID = set([x.split('-')[0] for x in os.listdir(TEST) if x.endswith('.jpg')])
missing = Patient_ID - set(pivoted.Patient_ID)
for item in missing:
    pivoted.loc[pivoted.shape[0], 'Patient_ID'] = item

columns = pd.read_csv(PATH).columns
columns = [x for x in columns if 'E' in x]
pivoted = pivoted.loc[:,['Patient_ID'] + columns]
pivoted = pivoted.fillna(2)

[2020-05-09 13:57:36,874][ERROR]   There is a mislabel LH_wrist_E__ulna for image UAB001-RF-22.jpg
[2020-05-09 13:57:36,876][ERROR]   There is a mislabel LH_wrist_E__ulna for image UAB002-RF-22.jpg
[2020-05-09 13:57:36,883][WARNING]   Number of missing per joint: {}
/home/keane/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:1418: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#deprecate-loc-reindex-listlike
  return self._getitem_tuple(key)


Joint,Patient_ID,LH_mcp_E__ip,LH_pip_E__2,LH_pip_E__3,LH_pip_E__4,LH_pip_E__5,LH_mcp_E__1,LH_mcp_E__2,LH_mcp_E__3,LH_mcp_E__4,...,LF_mtp_E__2,LF_mtp_E__3,LF_mtp_E__4,LF_mtp_E__5,RF_mtp_E__ip,RF_mtp_E__1,RF_mtp_E__2,RF_mtp_E__3,RF_mtp_E__4,RF_mtp_E__5
0,UAB001,2.0,0.0,0.0,3.0,2.0,0.0,1.0,2.0,2.0,...,0.0,0.0,2.0,3.0,2.0,2.0,0.0,3.0,4.0,0.0
1,UAB002,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0


In [25]:
pivoted.RF_mtp_E__ip

0   NaN
1   NaN
Name: RF_mtp_E__ip, dtype: float64

In [15]:
pivoted.to_csv(SCOREOUTPUT,index = False)
logger.info(f'Wrote erosion output to :{SCOREOUTPUT}!')

[2020-05-05 23:02:15,520][INFO]   Wrote erosion output to :./erosion_score.csv!
